# Análisis predictivo de supervivencia en el Titanic utilizando Snowflake ML

Este proyecto demuestra un ciclo completo de MLOps implementado en Snowflake, utilizando el conocido dataset del Titanic. El objetivo es predecir la supervivencia de los pasajeros utilizando características demográficas y del viaje.

A través de este proyecto, mostraremos cómo:
- Configurar y gestionar entornos de datos en Snowflake.
- Importar y preprocesar datos utilizando Snowpark.
- Desarrollar pipelines de transformación escalables.
- Entrenar y optimizar modelos con Snowflake ML.
- Evaluar modelos y registrarlos en un repositorio.
- Implementar modelos para inferencia en producción.

Todo el flujo de trabajo se ejecuta dentro del ecosistema de Snowflake, aprovechando su procesamiento distribuido y sus capacidades de ML integradas.

In [ ]:
# Snowpark y Snowflake
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col, lit, when, round, count, avg, cast
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType, LongType, DoubleType

# Snowflake ML para modelado
import snowflake.ml.modeling.preprocessing as snowml_preprocessing
import snowflake.ml.modeling.preprocessing as snowml
import snowflake.ml.modeling.impute as snowml_impute
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LogisticRegression 
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.ensemble import RandomForestClassifier # por si se quiere experimentar con otros modelos
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.modeling.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
from snowflake.ml.registry import Registry

# Bibliotecas para visualización y análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import shap

# Otras bibliotecas útiles
import json
import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Obtenemos la sesión activa
session = snowpark.Session.get_active_session()

# Configuramos el warehouse, database y schema
session.sql("USE WAREHOUSE TITANIC_ML_WH").collect()
session.sql("USE DATABASE TITANIC_ML_DB").collect()
session.sql("USE SCHEMA TITANIC_ML_SCHEMA").collect()

# Añadimos un query tag para análisis
session.query_tag = {"origin": "titanic_ml_demo", 
                     "name": "titanic_survival_prediction", 
                     "version": {"major": 1, "minor": 0},
                     "attributes": {"is_demo": 1}}

# Mostramos los detalles de la conexión
print('Conexión establecida con los siguientes parámetros:')
print(f'Usuario     : {session.get_current_user()}')
print(f'Rol         : {session.get_current_role()}')
print(f'Database    : {session.get_current_database()}')
print(f'Schema      : {session.get_current_schema()}')
print(f'Warehouse   : {session.get_current_warehouse()}')

In [ ]:
# Nota: Asumimos que el archivo CSV ya ha sido cargado en el stage TITANIC_ASSETS

# Crear un Snowpark DataFrame que cargue los datos desde el CSV del stage
titanic_df = session.read.options({
    "field_delimiter": ",",
    "field_optionally_enclosed_by": '"',
    "skip_header": 0,
    "parse_header": True,
    "infer_schema": True
}).csv("@TITANIC_ASSETS/titanic_data.csv")

# Mostrar las primeras filas del dataset
titanic_df.show(10)
titanic_df.describe() #observamos la distribución de las variables


print(titanic_df.columns) #tenemos las columnas entrecomilladas y hará el proceso de tratamiento de datos más tedioso
# Creamos la funcion que nos permite lidiar con ello
def clean_column_names(df, chars_to_remove='"', replace_with=''):
    """Elimina caracteres no deseados de los nombres de columnas de un Snowpark DataFrame"""
    for colname in df.columns:
        new_colname = colname
        for char in chars_to_remove:
            new_colname = new_colname.replace(char, replace_with)
        if new_colname != colname:
            df = df.with_column_renamed(colname, new_colname)
    return df
titanic_df = clean_column_names(titanic_df)
    
#Modificamos los nulos que importamos por defecto del conjunto del stage
titanic_df = titanic_df.withColumn(
        "AGE",
        when(col("AGE").is_null(), np.nan).otherwise(col("AGE"))
    )
#titanic_df.dtypes #observamos que embarked es importado como un double por lo que hacemos un tipecasting a string
titanic_df= titanic_df.withColumn("EMBARKED_CAST", col("EMBARKED").cast(StringType(2)))
titanic_df = titanic_df.withColumn(
        "EMBARKED_CAST",
        when(col("EMBARKED_CAST").is_null(), np.nan).otherwise(col("EMBARKED_CAST"))
    )
titanic_df = titanic_df.drop("EMBARKED_CAST")
print(titanic_df.columns)
print("\nGuardando el dataframe original en una tabla...")
titanic_df.write.mode('overwrite').save_as_table('TITANIC_ORIGINAL')


# Estrategia de preprocesamiento de datos con Snowpark ML

El manejo de datos faltantes y la transformación de variables es crucial para el rendimiento de los modelos predictivos. En las celdas siguientes, demostraremos cómo utilizar los distintos imputadores y transformadores que ofrece Snowflake ML:

- **SimpleImputer**: Para manejar valores faltantes en variables numéricas (como la edad) y categóricas (como el puerto de embarque).
- **OrdinalEncoder**: Para transformar variables categóricas ordinales como la clase del pasajero.
- **OneHotEncoder**: Para convertir variables categóricas nominales como el sexo y el puerto de embarque en formato numérico.
- **StandardScaler**: Para normalizar variables numéricas como la tarifa.

Aunque mostraremos cada transformación de forma independiente para fines didácticos, posteriormente encapsularemos toda esta lógica en una pipeline de preprocesamiento completa. Esta pipeline podrá ser reutilizada para procesar nuevos datos de manera consistente y automatizada.

In [ ]:
#simple imputer de snowpark.ml
try:
    frequent_imputer = snowml_impute.SimpleImputer(
        input_cols=["EMBARKED"],
        output_cols=["EMBARKED_IMPUTED"],
        strategy="most_frequent",
        missing_values=np.nan
    )
    print("\nSimpleImputer creado correctamente")
    transformed_titanic_df = frequent_imputer.fit(titanic_df).transform(titanic_df)
    print("\nSimpleImputer aplicado correctamente")
    
except Exception as e:
    print(f"Error al crear y ejecutar el SimpleImputer: {str(e)}")

try:
    simple_imputer = snowml_impute.SimpleImputer(
        input_cols=["AGE"],
        output_cols=["AGE_IMPUTED"],
        strategy="median",
        missing_values=np.nan
    )
    
    # Aplicamos el imputer
    print("\nSimpleImputer creado correctamente")
    transformed_titanic_df = simple_imputer.fit(transformed_titanic_df).transform(transformed_titanic_df)
    print("\nSimpleImputer aplicado correctamente")
    
except Exception as e:
    print(f"Error al crear y ejecutar el SimpleImputer: {str(e)}")

In [ ]:
#ordinal encoder de snowpark.ml
try:
    # Ajustamos las categorías según los valores actuales
    categories = {
        "PCLASS": np.array(["1", "2", "3"]) #Es importante definir las categorias de la variable en funcion del tiempo de dato que tiene la variable, puedes comprobar el tipo con df.dtypes
    }
    ordinal_encoder = snowml.OrdinalEncoder(
        input_cols=["PCLASS"],
        output_cols=["PCLASS_OE"],
        categories=categories
    )
    print("\nOrdinalEncoder creado correctamente")
    transformed_titanic_df = ordinal_encoder.fit(transformed_titanic_df).transform(transformed_titanic_df)
    print("\nOrdinalEncoder aplicado correctamente")
except Exception as e:
    print(f"Error al crear y ejecutar el OrdinalEncoder: {str(e)}")


In [ ]:
#onehot encoder de snowpark.ml
try:
    onehot_encoder = snowml.OneHotEncoder (
        input_cols=["SEX","EMBARKED_IMPUTED"],
        output_cols=["SEX_OHE","EMBARKED_OHE"],
        drop = "first",
        handle_unknown='ignore'
    )
    print("\nOnehotEncoder creado correctamente")
    transformed_titanic_df = onehot_encoder.fit(transformed_titanic_df).transform(transformed_titanic_df)
    print("\nOnehotEncoder aplicado correctamente")
except Exception as e:
    print(f"Error al crear y ejecutar el OrdinalEncoder: {str(e)}")

In [ ]:
#estandarización de snowpark.ml
try:
    standard_scaler = snowml.StandardScaler(
        input_cols=["FARE"],
        output_cols=["FARE_SCALED"],
        with_mean=True,  
        with_std=True  
    )
    print("\nStandardScaler creado correctamente")
    transformed_titanic_df = standard_scaler.fit(transformed_titanic_df).transform(transformed_titanic_df)
    print("\nStandardScaler aplicado correctamente")
except Exception as e:
    print(f"Error al crear y ejecutar el StandardScaler: {str(e)}")

In [ ]:
#KNN imputer de snowpark.ml, lo dejo en el código por motivos ilustrativos, no formará parte de la pipeline posterior
# try:
#     knn_imputer = snowml_impute.KNNImputer(
#         input_cols=["PCLASS_OE", "FARE_SCALED", "SIBSP", "PARCH"],
#         output_cols=["AGE_IMPUTED"],
#         n_neighbors=7,
#         weights="distance",
#         metric="nan_euclidean",
#         add_indicator=True
#     )
#     
#     print("\nKNNImputer creado correctamente")
#     transformed_titanic_df = knn_imputer.fit(transformed_titanic_df).transform(transformed_titanic_df)
#     print("\nKNNImputer aplicado correctamente")
#     
#     null_count_before = transformed_titanic_df.filter(col("AGE").is_null()).count()
#     null_count_after = transformed_titanic_df.filter(col("AGE_IMPUTED").is_null()).count()
#     print(f"Valores nulos en AGE antes: {null_count_before}")
#     print(f"Valores nulos en AGE_IMPUTED después: {null_count_after}")
#     
# except Exception as e:
#     print(f"Error al crear y ejecutar el KNNImputer: {str(e)}")

In [ ]:
columnas_a_eliminar = ["NAME", "TICKET", "PASSENGERID","CABIN","AGE","FARE","EMBARKED","SEX"]  

# Verificamos que existan todas las columnas que intentamos eliminar
columnas_existentes = transformed_titanic_df.columns
columnas_a_eliminar_filtradas = [col for col in columnas_a_eliminar if col in columnas_existentes]

# Eliminamos las columnas
transformed_titanic_df = transformed_titanic_df.drop(*columnas_a_eliminar_filtradas)
print(f"Columnas eliminadas: {columnas_a_eliminar_filtradas}")

# Mostramos las columnas finales
print("\nColumnas finales para modelado:")
print(transformed_titanic_df.columns)

# Pipeline de preprocesamiento: Automatización y reproducibilidad

Una ventaja clave de utilizar Snowflake ML es la capacidad de encapsular todo el preprocesamiento en una pipeline unificada. Esto garantiza:

1. **Reproducibilidad**: El mismo procesamiento se aplica a datos de entrenamiento y producción.
2. **Eficiencia**: Las transformaciones se ejecutan de forma distribuida en el warehouse de Snowflake.
3. **Mantenibilidad**: Es más fácil gestionar y actualizar un pipeline unificado que transformaciones independientes.
4. **Portabilidad**: La pipeline puede ser serializada, guardada en un stage y reutilizada según sea necesario.

En esta sección, construimos una pipeline que integra todas las transformaciones anteriores en un solo flujo de trabajo y la guardamos para su uso futuro con nuevos datos. Esto es fundamental para implementar soluciones de ML robustas en entornos de producción.

In [ ]:
print("\nCreando la pipeline de preprocesamiento...")

# Crear la pipeline
preprocessing_pipeline = Pipeline(
    steps=[
        # 1. Imputación para variables con valores faltantes
        (
            "embarked_imputer",
            snowml_impute.SimpleImputer(
                input_cols=["EMBARKED"],
                output_cols=["EMBARKED_IMPUTED"],
                strategy="most_frequent",
                missing_values=np.nan
            )
        ),
        (
            "age_imputer",
            snowml_impute.SimpleImputer(
                input_cols=["AGE"],
                output_cols=["AGE_IMPUTED"],
                strategy="median",
                missing_values=np.nan
            )
        ),

        (
            "ordinal_encoder",
            snowml.OrdinalEncoder(
                input_cols=["PCLASS"],
                output_cols=["PCLASS_OE"],
                categories=categories
            )
        ),

        (
            "onehot_encoder",
            snowml.OneHotEncoder(
                input_cols=["SEX", "EMBARKED_IMPUTED"],
                output_cols=["SEX_OHE", "EMBARKED_OHE"],
                drop="first",
                handle_unknown='ignore'
            )
        ),

        (
            "standard_scaler",
            snowml.StandardScaler(
                input_cols=["FARE"],
                output_cols=[ "FARE_SCALED"],
                with_mean=True,
                with_std=True
            )
        )
    ]
)


print("\nAjustando la pipeline a los datos...")
fitted_pipeline = preprocessing_pipeline.fit(titanic_df)
print("\nTransformando los datos con la pipeline...")
transformed_df = fitted_pipeline.transform(titanic_df)
print("\nPrimeras filas del dataframe transformado:")
transformed_df.show(5)
print("\nGuardando la pipeline en un archivo joblib...")
PIPELINE_FILE = '/tmp/titanic_preprocessing_pipeline.joblib'
joblib.dump(fitted_pipeline, PIPELINE_FILE)
print("\nSubiendo la pipeline al stage de Snowflake...")
session.file.put(PIPELINE_FILE, "@TITANIC_ASSETS", overwrite=True)

print("\n¡Pipeline creada, guardada y aplicada con éxito!")
print("La pipeline está disponible en: @TITANIC_ASSETS/titanic_preprocessing_pipeline.joblib")
print("\nGuardando los datos transformados en una tabla...")
transformed_df.write.mode('overwrite').save_as_table('TITANIC_TRANSFORMED')

print("\nDatos transformados guardados en la tabla: TITANIC_TRANSFORMED")


In [ ]:
#Una vez guardada, podemos aplicar la pipeline sobre titanic_original, obviando el código anterior
print("\nCargando la pipeline de preprocesamiento...")
session.file.get('@TITANIC_ASSETS/titanic_preprocessing_pipeline.joblib', '/tmp')
titanic_df = session.table("TITANIC_ORIGINAL")
loaded_pipeline = joblib.load('/tmp/titanic_preprocessing_pipeline.joblib')
print("Pipeline cargada correctamente")
print("\nAplicando la pipeline a los datos...")
transformed_df = loaded_pipeline.transform(titanic_df)
print("\nPrimeras filas del dataframe transformado:")
transformed_df.show(5)

# Análisis exploratorio: Descubriendo patrones en los datos del Titanic

El análisis exploratorio de datos (EDA) es fundamental para comprender las relaciones entre variables y guiar el desarrollo del modelo. En el caso del Titanic, algunas tendencias bien documentadas incluyen:

- **Género**: Las mujeres tuvieron tasas de supervivencia significativamente más altas que los hombres.
- **Clase**: Los pasajeros de primera clase sobrevivieron en mayor proporción que los de tercera.
- **Edad**: Los niños tuvieron prioridad en los botes salvavidas.
- **Tamaño familiar**: Personas viajando solas o en familias muy numerosas tuvieron menor probabilidad de supervivencia.

El gráfico generado muestra estas relaciones y otras más, proporcionando una visión integral de los factores que influyeron en la supervivencia.

In [ ]:
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("viridis")


transformed_df = session.table("TITANIC_TRANSFORMED")
original_df = session.table("TITANIC_ORIGINAL")
df = transformed_df.to_pandas()
df_original = original_df.to_pandas()

fig = plt.figure(figsize=(24, 24))

# 1. Supervivencia
plt.subplot(3, 3, 1)
survival_counts = df['SURVIVED'].value_counts(normalize=True) * 100
survival_abs = df['SURVIVED'].value_counts()
colors = ['#ff6666', '#5599ff']
plt.pie(survival_counts, labels=[f'No ({survival_abs[0]})', f'Sí ({survival_abs[1]})'], 
        autopct='%1.1f%%', startangle=90, colors=colors, explode=(0.05, 0.1),
        wedgeprops=dict(width=0.6, edgecolor='w'))
plt.title('Distribución de Supervivencia', fontsize=16)

# 2. Correlación
plt.subplot(3, 3, 2)
numeric_cols = ['SURVIVED', 'PCLASS_OE', 'AGE_IMPUTED', 'FARE_SCALED', 'SIBSP', 'PARCH']
for col in df.columns:
    if 'SEX_OHE' in col or 'EMBARKED_OHE' in col:
        numeric_cols.append(col)

corr = df[numeric_cols].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap=cmap, linewidths=0.7, 
            vmin=-1, vmax=1, annot_kws={"size": 9})
plt.title('Correlación entre Variables', fontsize=16)

# 3. Scatter plot Edad-Tarifa-Supervivencia-Clase
plt.subplot(3, 3, 3)
scatter = plt.scatter(df['AGE_IMPUTED'], df['FARE_SCALED'],
                      c=df['SURVIVED'].map({0: '#ff6666', 1: '#5599ff'}),
                      s=df['PCLASS_OE'].map({0: 150, 1: 100, 2: 50}),
                      alpha=0.7, edgecolors='w', linewidth=0.5)

plt.xlabel('Edad Imputada')
plt.ylabel('Tarifa Escalada')
plt.title('Relación Edad-Tarifa-Supervivencia-Clase', fontsize=16)

survival_legend = plt.legend(['No sobrevivió', 'Sobrevivió'], loc='upper right', title='Supervivencia')
plt.gca().add_artist(survival_legend)

class_sizes = [150, 100, 50]
class_labels = ['1ª Clase', '2ª Clase', '3ª Clase']
for i, size in enumerate(class_sizes):
    plt.scatter([], [], s=size, c='gray', alpha=0.7, edgecolors='w', linewidth=0.5, 
                label=class_labels[i])
plt.legend(scatterpoints=1, title='Clase', loc='upper left')

# 4. Violin plot de edad por género y supervivencia
plt.subplot(3, 3, 4)
if 'SEX_OHE_male' in df.columns:
    df['GENDER'] = df['SEX_OHE_male'].map({1: 'Hombre', 0: 'Mujer'})
elif 'SEX_OHE_female' in df.columns:
    df['GENDER'] = df['SEX_OHE_female'].map({1: 'Mujer', 0: 'Hombre'})
else:
    df['GENDER'] = df_original['SEX']

df['SURVIVED_LABEL'] = df['SURVIVED'].map({0: 'No', 1: 'Sí'})
violin = sns.violinplot(x='GENDER', y='AGE_IMPUTED', hue='SURVIVED_LABEL', 
                      data=df, palette=['#ff6666', '#5599ff'], 
                      split=True, inner='quart', linewidth=1)
violin.set_title('Distribución de Edad por Género y Supervivencia', fontsize=16)
violin.set_xlabel('Género')
violin.set_ylabel('Edad')

for gender in ['Hombre', 'Mujer']:
    for survived in [0, 1]:
        subset = df[(df['GENDER'] == gender) & (df['SURVIVED'] == survived)]
        if not subset.empty:
            x = 0 if gender == 'Hombre' else 1
            x_offset = -0.2 if survived == 0 else 0.2
            plt.annotate(f'{len(subset)}',
                        xy=(x + x_offset, subset['AGE_IMPUTED'].median()),
                        xytext=(0, 10), textcoords='offset points',
                        ha='center', va='bottom', fontsize=9,
                        bbox=dict(boxstyle='round,pad=0.2', fc='white', alpha=0.8))

# 5. Heatmap supervivencia por clase y género
plt.subplot(3, 3, 5)
class_gender_survival = pd.crosstab(
    [df['PCLASS_OE'].map({0: '1ª Clase', 1: '2ª Clase', 2: '3ª Clase'}), df['GENDER']],
    df['SURVIVED_LABEL'],
    normalize='index'
).reset_index()

class_gender_survival_pivot = class_gender_survival.pivot(
    index='PCLASS_OE', columns='GENDER', values='Sí'
)

sns.heatmap(class_gender_survival_pivot, annot=True, fmt='.1%', cmap='RdBu_r',
           linewidths=0.5, vmin=0, vmax=1, cbar_kws={'label': 'Tasa de Supervivencia'})
plt.title('Supervivencia por Clase y Género', fontsize=16)

# 6. Análisis de título y supervivencia
plt.subplot(3, 3, 6)
if 'NAME' in df_original.columns:
    df_original['TITLE'] = df_original['NAME'].str.extract(' ([A-Za-z]+)\.', expand=False)
    title_mapping = {
        'Capt': 'Officer', 'Col': 'Officer', 'Major': 'Officer', 'Dr': 'Professional',
        'Rev': 'Professional', 'Jonkheer': 'Royalty', 'Don': 'Royalty', 
        'Sir': 'Royalty', 'Lady': 'Royalty', 'the Countess': 'Royalty',
        'Dona': 'Royalty', 'Mme': 'Mrs', 'Mlle': 'Miss', 'Ms': 'Miss'
    }
    df_original['TITLE'] = df_original['TITLE'].map(lambda x: title_mapping.get(x, x))
    
    title_survival = df_original.groupby('TITLE')['SURVIVED'].agg(['mean', 'count']).reset_index()
    title_survival.columns = ['Título', 'Supervivencia', 'Cantidad']
    title_survival = title_survival[title_survival['Cantidad'] > 5].sort_values('Supervivencia', ascending=False)
    
    bars = sns.barplot(x='Título', y='Supervivencia', data=title_survival, 
                    palette=sns.color_palette("RdBu_r", len(title_survival)))
    
    for i, row in title_survival.iterrows():
        plt.text(i, row['Supervivencia']+0.02, f"{row['Supervivencia']*100:.1f}%\n({row['Cantidad']})", 
                ha='center', va='bottom', fontsize=9)
    
    plt.title('Supervivencia por Título', fontsize=16)
    plt.xlabel('Título')
    plt.ylabel('Tasa de Supervivencia')
    plt.ylim(0, 1.1)
    plt.xticks(rotation=45)

# 7. Análisis de puerto, clase y supervivencia
plt.subplot(3, 3, 7)
embarked_cols = [col for col in df.columns if 'EMBARKED_OHE' in col]

if embarked_cols:
    df['EMBARKED'] = None
    for col in embarked_cols:
        port = col.replace('EMBARKED_OHE_', '')
        df.loc[df[col] == 1, 'EMBARKED'] = port
    
    embarked_class_survival = pd.crosstab(
        [df['EMBARKED'], df['PCLASS_OE'].map({0: '1ª', 1: '2ª', 2: '3ª'})],
        df['SURVIVED'],
        normalize='index'
    )[1].unstack()
    
    sns.heatmap(embarked_class_survival, annot=True, fmt='.1%', cmap='RdBu_r',
               linewidths=0.5, vmin=0, vmax=1, cbar_kws={'label': 'Tasa de Supervivencia'})
    plt.title('Supervivencia por Puerto y Clase', fontsize=16)

# 8. Relación entre tamaño familiar y supervivencia
plt.subplot(3, 3, 8)
df['FAMILY_SIZE'] = df['SIBSP'] + df['PARCH'] + 1
df['FAMILY_GROUP'] = pd.cut(df['FAMILY_SIZE'], 
                            bins=[0, 1, 2, 4, 11], 
                            labels=['Solo', 'Pequeña (2)', 'Mediana (3-4)', 'Grande (5+)'])

fam_survival = df.groupby('FAMILY_GROUP')['SURVIVED'].agg(['mean', 'count']).reset_index()
fam_survival.columns = ['Tamaño Familiar', 'Supervivencia', 'Cantidad']

ax = plt.subplot(3, 3, 8)
fam_survival['se'] = np.sqrt((fam_survival['Supervivencia'] * (1 - fam_survival['Supervivencia'])) / fam_survival['Cantidad'])
fam_survival['ci'] = fam_survival['se'] * 1.96

bars = plt.bar(fam_survival['Tamaño Familiar'], fam_survival['Supervivencia'], 
              yerr=fam_survival['ci'], capsize=5, 
              color=sns.color_palette("viridis", len(fam_survival)))

for i, bar in enumerate(bars):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03, 
            f"{fam_survival.iloc[i]['Supervivencia']*100:.1f}%\n({fam_survival.iloc[i]['Cantidad']})", 
            ha='center', va='bottom', fontsize=10)

plt.title('Supervivencia por Tamaño Familiar', fontsize=16)
plt.xlabel('Grupo Familiar')
plt.ylabel('Tasa de Supervivencia')
plt.ylim(0, 1.1)

# 9. Análisis de clase, edad y supervivencia
plt.subplot(3, 3, 9)
df['AGE_GROUP'] = pd.cut(df['AGE_IMPUTED'], 
                        bins=[0, 12, 18, 35, 50, 100],
                        labels=['Niño (<12)', 'Adolescente (12-18)', 'Joven (19-35)', 'Adulto (36-50)', 'Mayor (>50)'])

age_class_survival = pd.crosstab(
    [df['AGE_GROUP'], df['PCLASS_OE'].map({0: '1ª', 1: '2ª', 2: '3ª'})],
    df['SURVIVED'],
    normalize='index'
)[1].unstack()

sns.heatmap(age_class_survival, annot=True, fmt='.1%', cmap='RdBu_r',
           linewidths=0.5, vmin=0, vmax=1, cbar_kws={'label': 'Tasa de Supervivencia'})
plt.title('Supervivencia por Grupo de Edad y Clase', fontsize=16)

plt.tight_layout(pad=3.0)
plt.subplots_adjust(top=0.93)
plt.suptitle('ANÁLISIS EXPLORATORIO TITANIC', fontsize=24, y=0.98)

plt.savefig('/tmp/titanic_eda_advanced.png', dpi=300, bbox_inches='tight')
try:
    session.file.put('/tmp/titanic_eda_advanced.png', '@TITANIC_ASSETS', overwrite=True)
except:
    pass

# Modelado predictivo: Comparativa y optimización de modelos

En esta sección, desarrollaremos dos modelos de clasificación para predecir la supervivencia:

1. **Modelo base**: Implementación directa de XGBoost con parámetros por defecto.
2. **Modelo optimizado**: Versión mejorada mediante búsqueda de hiperparámetros con GridSearchCV.

Compararemos ambos modelos utilizando métricas estándar:
- **Exactitud (Accuracy)**: Proporción general de predicciones correctas.
- **Puntuación F1 (F1 Score)**: Media armónica de precisión y exhaustividad.
- **Área bajo la curva ROC (ROC AUC)**: Capacidad del modelo para distinguir entre clases.

Ambos modelos serán registrados en el repositorio de modelos de Snowflake para:
- Mantener un historial de versiones.
- Documentar métricas de rendimiento.
- Facilitar la implementación en producción.

Finalmente, seleccionaremos el modelo con mejor rendimiento para la inferencia de nuevos datos.

In [ ]:
# Cargamos los datos transformados
titanic_transformed_df = session.table("TITANIC_TRANSFORMED")
print("Primeras filas del dataset transformado:")
titanic_transformed_df.show(5)
titanic_transformed_df = clean_column_names(titanic_transformed_df)


CATEGORICAL_COLUMNS = ["PCLASS_OE", "SEX_OHE_MALE", "EMBARKED_OHE_Q","EMBARKED_OHE_S"]
NUMERICAL_COLUMNS = ["AGE_IMPUTED", "FARE_SCALED", "SIBSP", "PARCH"]
INPUT_COLUMNS = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS



TARGET_COLUMN = ["SURVIVED"]
OUTPUT_COLUMN = ["PREDICTION"]
print("\nColumnas de entrada para el modelo:")
print(INPUT_COLUMNS)
print("\nColumna objetivo:")
print(TARGET_COLUMN)

In [ ]:
#Dividimos el conjunto de datos en entrenamiento y prueba
train_df, test_df = titanic_transformed_df.random_split(weights=[0.8, 0.2], seed=42)

print(f"Tamaño del conjunto de entrenamiento: {train_df.count()}")
print(f"Tamaño del conjunto de testeo: {test_df.count()}")
print("\nDistribución de supervivencia en el conjunto de entrenamiento:")
train_df.group_by("SURVIVED").count().show()
print("\nDistribución de supervivencia en el conjunto de testeo:")
test_df.group_by("SURVIVED").count().show()

In [ ]:
# Entrenamos un Modelo Simple sin tuneo de hiperparámetros ni nada sofisticado 
xgb_classifier = XGBClassifier(
    input_cols=INPUT_COLUMNS,
    label_cols=TARGET_COLUMN,
    output_cols=OUTPUT_COLUMN
)

print("Entrenando el modelo XGBoost...")
xgb_classifier.fit(train_df)
result = xgb_classifier.predict(test_df)

# Evaluar el modelo
accuracy = accuracy_score(df=result, y_true_col_names="SURVIVED", y_pred_col_names="PREDICTION")
f1 = f1_score(df=result, y_true_col_names="SURVIVED", y_pred_col_names="PREDICTION")
roc_auc = roc_auc_score(
    df=result, 
    y_true_col_names="SURVIVED", 
    y_score_col_names="PREDICTION", 
    average='macro'
)

print(f"\nResultados del modelo simple:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# Mostrar matriz de confusión con parámetros ajustados
conf_matrix = confusion_matrix(
    df=result, 
    y_true_col_name="SURVIVED",  
    y_pred_col_name="PREDICTION", 
    normalize=None
)
print("\nMatriz de Confusión:")
print(conf_matrix)

In [ ]:
# Exprimentamos con gridsearch y validación cruzada para la optimización de hiperparámetros
print("Escalando el warehouse...")
session.sql("ALTER WAREHOUSE TITANIC_ML_WH SET WAREHOUSE_SIZE=MEDIUM").collect()


param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2]
}
grid_search = GridSearchCV(
    estimator=XGBClassifier(),
    param_grid=param_grid,
    scoring="accuracy",
    n_jobs=-1,  # Utilizar todos los procesadores disponibles
    input_cols=INPUT_COLUMNS,
    label_cols=TARGET_COLUMN,
    output_cols=OUTPUT_COLUMN
)

print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(train_df)
best_params = grid_search.to_sklearn().best_params_
best_score = grid_search.to_sklearn().best_score_
print(f"\nMejores parámetros encontrados: {best_params}")
print(f"Mejor puntuación de validación cruzada: {best_score:.4f}")


session.sql("ALTER WAREHOUSE TITANIC_ML_WH SET WAREHOUSE_SIZE=XSMALL").collect()

In [ ]:
#Evaluamos el mejor modelo en el conjunto de testeo
best_model_results = grid_search.predict(test_df)

best_accuracy = accuracy_score(df=best_model_results, y_true_col_names="SURVIVED", y_pred_col_names="PREDICTION")
best_f1 = f1_score(df=best_model_results, y_true_col_names="SURVIVED", y_pred_col_names="PREDICTION")
best_roc_auc = roc_auc_score(
    df=best_model_results, 
    y_true_col_names="SURVIVED", 
    y_score_col_names="PREDICTION",
    average='macro'
)

print(f"\nResultados del mejor modelo:")
print(f"Accuracy: {best_accuracy:.4f}")
print(f"F1 Score: {best_f1:.4f}")
print(f"ROC AUC: {best_roc_auc:.4f}")
best_model_results_pd = best_model_results.to_pandas()

cm = confusion_matrix(
    df=best_model_results, 
    y_true_col_name="SURVIVED", 
    y_pred_col_name="PREDICTION",  
    normalize=None
)
print(cm)

In [ ]:
#Guardamos ambas versiones del modelo con registry y log_model
db = session.get_current_database()
schema = session.get_current_schema()
registry = Registry(session=session, database_name=db, schema_name=schema)
model_name = "TITANIC_SURVIVAL_PREDICTOR"

# Registrar el modelo simple (versión inicial)
model_ver_initial = registry.log_model(
    model_name=model_name,
    version_name="V3", #v3 y v4 porque ya hemos hecho este proceso 2 veces
    model=xgb_classifier,
    sample_input_data=test_df[INPUT_COLUMNS],
    options={"enable_explainability": True}
)
model_ver_initial.set_metric(metric_name="accuracy", value=accuracy)
model_ver_initial.set_metric(metric_name="f1_score", value=f1)
model_ver_initial.set_metric(metric_name="roc_auc", value=roc_auc)
model_ver_initial.comment = "Modelo inicial XGBoost para predecir supervivencia en el Titanic."

# Registrar el mejor modelo de GridSearchCV
model_ver_optimized = registry.log_model(
    model_name=model_name,
    version_name="V4",
    model=grid_search.to_sklearn().best_estimator_,
    sample_input_data=test_df[INPUT_COLUMNS],
    options={"enable_explainability": True}
)
model_ver_optimized.set_metric(metric_name="accuracy", value=best_accuracy)
model_ver_optimized.set_metric(metric_name="f1_score", value=best_f1)
model_ver_optimized.set_metric(metric_name="roc_auc", value=best_roc_auc)
model_ver_optimized.set_metric(metric_name="best_params", value=str(best_params))
model_ver_optimized.comment = "Modelo XGBoost optimizado con GridSearchCV para predecir supervivencia en el Titanic."


print("\nModelos registrados:")
registry.get_model(model_name).show_versions()

In [ ]:
#Utilizamos el modelo guardado para realizar inferencia (celda independiente una vez ya se han registrado los modelos)
db = session.get_current_database()
schema = session.get_current_schema()
registry = Registry(session=session, database_name=db, schema_name=schema)
model_name = "TITANIC_SURVIVAL_PREDICTOR"
train_df, test_df = titanic_transformed_df.random_split(weights=[0.8, 0.2], seed=42)
optimized_model = registry.get_model(model_name).version("V4")


sample_data = test_df.limit(5)
print("Datos de ejemplo para inferencia:")
sample_data.show()

# Realizar predicciones
predictions = optimized_model.run(sample_data, function_name="predict")
predictions = clean_column_names(predictions)
    
print("\nPredicciones:")
predictions.select("SURVIVED", "OUTPUT_FEATURE_0").show()

In [ ]:
# Aportamos explicabilidad del modelo

# Generar explicaciones para algunas muestras
explanations = optimized_model.run(test_df.limit(50), function_name="explain")
print("Explicaciones del modelo:")
explanations.show(5)
explanations = clean_column_names(explanations)
feature_cols = [col + "_explanation" for col in INPUT_COLUMNS]
explanations_pd = explanations.select(feature_cols).to_pandas()


shap_exp = shap._explanation.Explanation(
    values=explanations_pd.values, 
    feature_names=INPUT_COLUMNS
)
plt.close('all')
plt.figure(figsize=(5, 5))
shap.plots.bar(shap_exp, show=False) 
plt.title('Importancia absoluta de las Características en la Predicción de Supervivencia')
plt.tight_layout()
plt.show() 

In [ ]:
# Exportación del modelo para uso externo (también se puede hacer desde el stage)
import os
model_export_dir = '/tmp/titanic_model'
if not os.path.exists(model_export_dir):
    os.makedirs(model_export_dir)

optimized_model.export(model_export_dir)
print(f"Modelo exportado a: {model_export_dir}")

# Cargar el modelo para verificar
loaded_model = optimized_model.load()
print("Modelo cargado correctamente")